https://www.kaggle.com/arthurtok/introduction-to-ensembling-stacking-in-python
위의 주소의 커널을 해석하였습니다. 이 글이 맘에 드신다면 위의 링크로 가셔서 upvote버튼을 눌러주세요.

# Introduction


이 노트는 기본 학습 모델을 앙상블 (결합)하는 방법, 특히 스태킹 (stacking)으로 알려진 앙상블의 변형에 대한 매우 기본적이고 간단한 소개 입문서입니다. 
간단히 말해서 스태킹은 첫 번째단계에서 몇 가지 기본 분류모델의 예측을 사용한다음 두 번째 level에서 다른 모델을 사용하여 이전 수준의 첫 번째 예측 결과를 예측하는 것입니다.

타이타닉 데이터셋은 캐글을 접한지 얼마 안되는 초보자들에게 이 개념(Stacking)을 소개하는 좋은 데이터셋 중 하나입니다. 그러나 스태킹이 Kaggle 대회에서 많은 팀이 우승하도록 기여했음에도 불구하고 이 주제에 대한 커널이 부족한 것으로 보이므로 이 노트북이 그 공백을 조금이라도 채울 수 있기를 바랍니다.

나(kernel Writer)도 Kaggle에 온지 얼마 안 된 신참이고 우연히 만나고 공부할 수 있던 ensembling / stacking script는 훌륭한 Faron에 의한 the AllState Severity Claims competition에 쓰여있던것이었다.(Open 되어 있지 않고 공부하기 힘들었다 이런뜻인듯). 이 노트북에 쓰여진 자료들은 Faron의 script에서 많이 차용되었습니다. classifier의 앙상블이 여기에서 쓰였지만 그는 regressors의 앙상블을 이용했습니다.
어쨌든 그의 script를 한 번 확인하십시오:

[Stacking Starter][1] : by Faron 

이 notebook을 앞에 두고 앙상블의 개념을 직관적이고 간결한 방식으로 다루고 전달할 수 있기를 희망합니다. 나의 다른 독립적인 Kaggle [script][2]는 아래에서 논의되는 앙상블 단계를(다른 매개 변수임에도 불구하고) 정확하게 동일하게 수행하는데, 이 Kaggle [script][2]은 top 9%에 들기에 충분한 0.808의 LB 점수를 기록하고 4분 안에 코드가 실행됩니다. 따라서 나는 그 스크립트를 개선하고 추가 할 여지가 많다는 것을 확신합니다. 어쨌든 내가 어떻게 개선 할 수 있는지에 대한 의견을 남겨주세요.

 [1]: https://www.kaggle.com/mmueller/allstate-claims-severity/stacking-starter/run/390867
 [2]: https://www.kaggle.com/arthurtok/titanic/simple-stacking-with-xgboost-0-808

In [ ]:
# Load in our libraries
import pandas as pd
import numpy as np
import re
import sklearn
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

import warnings
warnings.filterwarnings('ignore')

# Going to use these 5 base models for the stacking
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn.svm import SVC
#from sklearn.cross_validation import KFold
from sklearn.model_selection import KFold

# Feature Exploration, Engineering and Cleaning 

이제 대부분의 커널이 구성되어있는 구조와 매우 비슷하게 진행될것입니다. 
<br>즉 먼저 데이터를 탐색하고, feature engineering 가능성을 확인하고 범주형 features를 numrically(수치형)으로 바꿔줄것 입니다. 

In [ ]:
# train 과 test 데이터셋을 불러드립니다.
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

#승객의 ID를 저장합니다 for easy access
PassengerId = test['PassengerId']

train.head(3)

당연하게도 우리가 할일은 범주형 변수에서 정보를 얻어내는것 입니다.

**Feature Engineering**

feature enginnering idea를 얻기 위해 매우 이해하기 쉽게 잘 알려주는 Sina's notebook에서 많은 것을 배웠습니다. 그러니 그의 작업을 한번 확인해보세요.

[Titanic Best Working Classfier][1] : by Sina


  [1]: https://www.kaggle.com/sinakhorami/titanic/titanic-best-working-classifier

In [ ]:
full_data = [train, test]

# 스스로 더한 몇몇의 feature
# 이름의 길이를 제공해준다.
train['Name_length'] = train['Name'].apply(len)
test['Name_length'] = test['Name'].apply(len)
# 타이타닉에 탄 승객이 Cabin의 유무에대한 feature
train['Has_Cabin'] = train["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
test['Has_Cabin'] = test["Cabin"].apply(lambda x: 0 if type(x) == float else 1)

# Sina가 수행한 Feature engineering 단계들
# Create new feature FamilySize as a combination of SibSp and Parch
for dataset in full_data:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
# 혼자인지 여부에 대한 새로운 feature 생성
for dataset in full_data:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1
# Embarked에 있는 결측치 모두 삭제
for dataset in full_data:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')
# Fare 열에 있는 결측치를 모두 삭제하고 새로운 CategoricalFare를 만든다.
for dataset in full_data:
    dataset['Fare'] = dataset['Fare'].fillna(train['Fare'].median())
train['CategoricalFare'] = pd.qcut(train['Fare'], 4)
# 새로운 CategoricalAge를 생성한다.
for dataset in full_data:
    age_avg = dataset['Age'].mean()
    age_std = dataset['Age'].std()
    age_null_count = dataset['Age'].isnull().sum()
    age_null_random_list = np.random.randint(age_avg - age_std, age_avg + age_std, size=age_null_count)
    dataset['Age'][np.isnan(dataset['Age'])] = age_null_random_list
    dataset['Age'] = dataset['Age'].astype(int)
train['CategoricalAge'] = pd.cut(train['Age'], 5)
# 승객에서 title를 알아내기 위한 함수를 정의한다.
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    # title이 존재한다면 그것을 추출하고 그 값을 반환한다.
    if title_search:
        return title_search.group(1)
    return ""
# 승객의 title을 포함하고 있는 새로운 feature인 Title을 생성한다.
for dataset in full_data:
    dataset['Title'] = dataset['Name'].apply(get_title)
# 흔하지 않은 title은 단일 그룹인 'Rare'로 포함시킨다.
for dataset in full_data:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

for dataset in full_data:
    # Mapping Sex
    dataset['Sex'] = dataset['Sex'].map( {'female': 0, 'male': 1} ).astype(int)
    
    # Mapping titles
    title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)
    
    # Mapping Embarked
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
    
    # Mapping Fare
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] 						        = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] 							        = 3
    dataset['Fare'] = dataset['Fare'].astype(int)
    
    # Mapping Age
    dataset.loc[ dataset['Age'] <= 16, 'Age'] 					       = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age'] = 4 ;

In [ ]:
# Feature selection
drop_elements = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp']
train = train.drop(drop_elements, axis = 1)
train = train.drop(['CategoricalAge', 'CategoricalFare'], axis = 1)
test  = test.drop(drop_elements, axis = 1)

이제 features를 정리했고 적절한 정보들을 얻고 범주형 column들을 삭제하였습니다. 이제 모든 features은 머신러닝 모델에 적합한 형식인 수치형(numeric)이어야 합니다. 다음 단계로 진행하기 전에 간단한 상관관계와 변환된 데이터셋의 분포 plot을 그려보자.
## Visualisations 

In [ ]:
train.head(3)

**Pearson Correlation Heatmap**

몇개의 feature들의 상관관계 plot을 생성해서 한 feature과 다른 feature와 연관되어 있는지 확인해 볼것입니다. 그렇게 하기 위해 Seaborn Plotting package를 활용할것 입니다. 이 package는 다음과 같이 heatmap을 매우 편리하게 나타내어 줍니다.

In [ ]:
colormap = plt.cm.RdBu
plt.figure(figsize=(14,12))
plt.title('Pearson Correlation of Features', y=1.05, size=15)
sns.heatmap(train.astype(float).corr(),linewidths=0.1,vmax=1.0, 
            square=True, cmap=colormap, linecolor='white', annot=True)

**Takeaway from the Plots**

the Pearson Correlation plot 을 통해 알 수 있는것은 많은 변수들이 서로간에 큰 상관성을 보이지 않는다는 것입니다. 이건 이 feature들을 당신의 learning model에 feeding(활용? 넣어준다?) 관점에서는 좋은 현상입니다. 왜냐면 이건 우리의 학습 데이터에 불필요하거나 지나치게 많은 데이터가 많지 않다는것을 의미하고 각각의 feature가 unique한 정보를 가지고 있다는 것입니다. 여기서 가장 강한 상관성을 보이는 것은 Family size 와 Parch(Parents 와 children)입니다. 여기서는 연습을 위한 목적으로 두 개의 feature 다 남겨 둘 것입니다.
.

**Pairplots**

마지막으로 하나의 피쳐에서 다른 피쳐로 데이터의 분포를 관찰하기 위해 몇 개의 pairplots을 생성하겠습니다. 다시 한번 이 작업을 위해 시본 (Seaborn)을 사용합니다.

In [ ]:
g = sns.pairplot(train[[u'Survived', u'Pclass', u'Sex', u'Age', u'Parch', u'Fare', u'Embarked',
       u'FamilySize', u'Title']], hue='Survived', palette = 'seismic',size=1.2,diag_kind = 'kde',diag_kws=dict(shade=True),plot_kws=dict(s=10) )
g.set(xticklabels=[])

# Ensembling & Stacking models

마침내 feature engineering 과 formatting에 관해 간단하고 매우 빠르게 진행된 과정을 통해, 우리는 이 notebook의 요지와 meat(목표?)에 도달했습니다.  

Stacking ensemble을 만듭시다!

### Helpers via Python Classes

여기에서는 파이썬 클래스를 사용하여보다 편리하게 사용할 수 있도록합니다. 프로그래밍을 처음 시작하는 사람은 일반적으로 객체 지향 프로그래밍 (OOP)과 함께 사용되는 클래스를 들어봤을것 입니다. 즉, 클래스는 객체(구시대 사람들을 위한 변수) 생성을위한 코드 / 프로그램 을 확장하고 해당 클래스에 특정한 함수 및 메소드를 구현하는 데 도움이됩니다.


아래의 코드 섹션에서는 모든 Sklearn classifiers에 공통적으로 내장 된 메서드 (예 : train, predict and fit)를 확
장 할 수있는 클래스 *SklearnHelper* 를 사용합니다. 따라서 5 개의 다른 classifiers를 호출하려면 동일한 메소드를 5 번 작성할 필요가 없으므로 불필요한 중복을 제거해 줍니다.

In [ ]:
# 나중에 유용하게 쓸 수있는 유용한 매개 변수
ntrain = train.shape[0]
ntest = test.shape[0]
SEED = 0 # 재현성을 위한 값
NFOLDS = 5 # out-of-fold prediction 위한 fold값 설정
kf = KFold(5, shuffle=True, random_state=0)

# Sklearn classifier 확장하기 위한 클래스
#**params https://stackoverflow.com/questions/36901/what-does-double-star-asterisk-and-star-asterisk-do-for-parameters
class SklearnHelper(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_state'] = seed
        self.clf = clf(**params) 

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict(x)
    
    def fit(self,x,y):
        return self.clf.fit(x,y)
    
    def feature_importances(self,x,y):
        print(self.clf.fit(x,y).feature_importances_)
    
# Class to extend XGboost classifer


파이썬에서 클래스나 객체를 만들어 보지 않은 사람들을 위해 이미 이것을 알고 있는 사람들은 조금만 참아주시기 바랍니다. 위에서 주어진 코드가 무엇을하는지 설명하겠습니다. 기본 classifiers 만들때 Sklearn 라이브러리에 이미있는 모델만 사용하므로 클래스를 확장 할 수 있습니다.

**def init** : 클래스의 기본 생성자를 호출하는 Python 표준입니다. 즉, 객체(classifier)를 만들려면 clf(원하는 sklearn classifier), seed(임의 시드) 및 params(parameters for the classifiers)의 매개 변수값을 지정해야합니다.

나머지 코드는 단순히 sklearn classifier안에 이미 존재하는 해당 메소드를 호출하는 클래스의 메소드입니다. 기본적으로 다양한 Sklearn 분류기를 확장하는 wrapper 클래스를 만들어 stacker에 여러 learners를 구현할 때 동일한 코드를 반복 작성해야하는 부담을 줄여줍니다.

### Out-of-Fold Predictions

이제 소개 섹션에서 언급 한 것처럼 스태킹은 기본 classifiers의 예측을 두 번째 레벨 모델에 대한 학습용 input값으로 사용합니다. 그러나 전체 학습 데이터에대해 기본 모델을 단순히 학습 할 수는 없습니다. 전체 테스트 세트에서 예측을 생성 한 뒤에 두 번째 레벨 모델의 학습을 위해 이 예측값을 입력해야 합니다. 이렇게하면 기본 모델 예측이 이미 테스트 데이터 셋을 보았다는 risk를 수반하므로 이 예측값을 feeding할 때 overfitting이 발생합니다.

In [ ]:
np.empty((5, 10))

In [ ]:
def get_oof(clf, x_train, y_train, x_test):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))

    for i, (train_index, test_index) in enumerate(kf.split(x_train)):
        x_tr = x_train[train_index]
        y_tr = y_train[train_index]
        x_te = x_train[test_index]

        clf.train(x_tr, y_tr)

        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(x_test) # i번째 fold 의 예측값

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

# Generating our Base First-Level Models 

자 이제 우리의 첫번째 단계의 classification으로 5개의 learning model을 준비합시다. 이 models은 Sklean library를 통해 쉽게 불러올 수 있습니다. 모델의 리스트는 다음과 같습니다:

 1. Random Forest classifier
 2. Extra Trees classifier
 3. AdaBoost classifer
 4. Gradient Boosting classifer
 5. Support Vector Machine

**Parameters**

완전성을 위해 여기에 나열 할 매개 변수에 대한 간단한 요약만 제공합니다.

**n_jobs** : 학습과정에 사용되는 cores의 수로 만약 -1로 설정되어 있다면 모든 core가 사용됩니다.

**n_estimators** : 당신의 learning model 안에 classification의 수(set to 10 per default)

**max_depth** : 트리의 깊이의 최대값 또는 얼마나 많은 노드를 확장할지에 대한 값으로 이것이 너무 큰 값을 가진다면 트리가 지나치게 깊게 자랄 수 있으므로 오버피팅의 위험을 수반할 것입니다.

**verbose** : 학습 과정 중에 텍스트를 출력할지 여부를 조정합니다. 값 0은 모든 텍스트를 억제하는 반면 값 3은 모든 반복에서 트리 학습 프로세스를 출력합니다.


Sklearn 공식 웹 사이트를 통해 자세한 설명을 확인하십시오. 거기서 모델에서 설정할 수 있는 다른 유용한 매개 변수의 전체 정보를 찾을 수 있을것입니다.


In [ ]:
# 언급한 classifier의 parameters를 넣어준다
# Random Forest parameters
rf_params = {
    'n_jobs': -1,
    'n_estimators': 500,
     'warm_start': True, 
     #'max_features': 0.2,
    'max_depth': 6,
    'min_samples_leaf': 2,
    'max_features' : 'sqrt',
    'verbose': 0
}

# Extra Trees Parameters
et_params = {
    'n_jobs': -1,
    'n_estimators':500,
    #'max_features': 0.5,
    'max_depth': 8,
    'min_samples_leaf': 2,
    'verbose': 0
}

# AdaBoost parameters
ada_params = {
    'n_estimators': 500,
    'learning_rate' : 0.75
}

# Gradient Boosting parameters
gb_params = {
    'n_estimators': 500,
     #'max_features': 0.2,
    'max_depth': 5,
    'min_samples_leaf': 2,
    'verbose': 0
}

# Support Vector Classifier parameters 
svc_params = {
    'kernel' : 'linear',
    'C' : 0.025
    }

또한, OOP 프레임 워크 내에서 객체와 클래스에 대해 언급 한 이후로, 앞서 정의한 Helper Sklearn 클래스를 통해 5 가지 학습 모델을 나타내는 5 개의 객체를 생성해 보겠습니다.

In [ ]:
# Create 5 objects that represent our 4 models
rf = SklearnHelper(clf=RandomForestClassifier, seed=SEED, params=rf_params)
et = SklearnHelper(clf=ExtraTreesClassifier, seed=SEED, params=et_params)
ada = SklearnHelper(clf=AdaBoostClassifier, seed=SEED, params=ada_params)
gb = SklearnHelper(clf=GradientBoostingClassifier, seed=SEED, params=gb_params)
svc = SklearnHelper(clf=SVC, seed=SEED, params=svc_params)

**train and test sets으로 부터 Numpy arrays를 생성**

첫 번째 레이어 base 모델을 준비한 다음에는 다음과 같이 원래 데이터 프레임에서 NumPy 배열을 생성하여 classifier의 input 에 대한 train 및 test 데이터를 준비 할 수 있습니다.

In [ ]:
# 우리의 모델로 값을 넣어주기 위한 train, test and target ( Survived) dataframes의 Numpy arrays 생성
y_train = train['Survived'].ravel()
train = train.drop(['Survived'], axis=1)
x_train = train.values # Creates an array of the train data
x_test = test.values # Creats an array of the test data

**첫번째 수준의 예측의 값** 


이제 5 개의 기본 classifier에 train and test data를 제공하고 앞서 정의한 Out-of-Fold 예측 함수를 사용하여 첫 번째 수준 예측을 생성합니다. 아래 코드를 실행하려면 몇 분 정도 기다려주십시오.

In [ ]:
# Create our OOF train and test predictions. These base results will be used as new features
et_oof_train, et_oof_test = get_oof(et, x_train, y_train, x_test) # Extra Trees
rf_oof_train, rf_oof_test = get_oof(rf,x_train, y_train, x_test) # Random Forest
ada_oof_train, ada_oof_test = get_oof(ada, x_train, y_train, x_test) # AdaBoost 
gb_oof_train, gb_oof_test = get_oof(gb,x_train, y_train, x_test) # Gradient Boost
svc_oof_train, svc_oof_test = get_oof(svc,x_train, y_train, x_test) # Support Vector Classifier

print("Training is complete")

**다른 classifiers에서 만들어진 Feature importance **

이제 첫 번째 수준의 classifiers를 학습시켰으므로, Sklearn 모델의 매우 멋진 기능을 활용할 수 있습니다. 이는 매우 간단한 코드 한 줄로 실행되어서 다양한 feature의 importance의 값을 출력해줍니다. 

Sklearn 문서에 따르면 대부분의 classifier에는 단순히 **.feature_importances**를 입력하여 feature importance를 반환해주는 특성이 내제되어 있습니다. 그러므로 우리는 이 매우 유용한 특성을 앞서 언급한 함수를 통해 호출할 것이고 feature importance를 다음과 같이 그래프로 나타낼 것입니다.

In [ ]:
rf_feature = rf.feature_importances(x_train,y_train)
et_feature = et.feature_importances(x_train, y_train)
ada_feature = ada.feature_importances(x_train, y_train)
gb_feature = gb.feature_importances(x_train,y_train)

feature importance값을 어떻게 할당하고 저장해야 하는지 아직 정확하게 파악하지 못했습니다. 그러므로 코드로 값을 출력하고 그 다음 아래와 같은 파이썬 리스트에 붙여넣기 하겠습니다. (sorry for the lousy hack)

In [ ]:
rf_features = [0.10474135,  0.21837029,  0.04432652,  0.02249159,  0.05432591,  0.02854371
  ,0.07570305,  0.01088129 , 0.24247496,  0.13685733 , 0.06128402]
et_features = [ 0.12165657,  0.37098307  ,0.03129623 , 0.01591611 , 0.05525811 , 0.028157
  ,0.04589793 , 0.02030357 , 0.17289562 , 0.04853517,  0.08910063]
ada_features = [0.028 ,   0.008  ,      0.012   ,     0.05866667,   0.032 ,       0.008
  ,0.04666667 ,  0.     ,      0.05733333,   0.73866667,   0.01066667]
gb_features = [ 0.06796144 , 0.03889349 , 0.07237845 , 0.02628645 , 0.11194395,  0.04778854
  ,0.05965792 , 0.02774745,  0.07462718,  0.4593142 ,  0.01340093]

Plotly 패키지를 통해 쉽게 그래프를 그릴 수 있도록 feature importance data가 포함된 list로 데이터 프레임을 만듭니다.

In [ ]:
cols = train.columns.values
# Create a dataframe with features
feature_dataframe = pd.DataFrame( {'features': cols,
     'Random Forest feature importances': rf_features,
     'Extra Trees  feature importances': et_features,
      'AdaBoost feature importances': ada_features,
    'Gradient Boost feature importances': gb_features
    })

**Plotly scatterplots통해 대화형 feature importance plotting**

이 시점에서 대화형 Plotly 패키지를 사용해서 plotly scatter plot 통해 "Scatter"을 호출하여 다른 classifier들의 feature importance를 시각화 하겠습니다  

In [ ]:
# Scatter plot 
trace = go.Scatter(
    y = feature_dataframe['Random Forest feature importances'].values,
    x = feature_dataframe['features'].values,
    mode='markers',
    marker=dict(
        sizemode = 'diameter',
        sizeref = 1,
        size = 25,
#       size= feature_dataframe['AdaBoost feature importances'].values,
        #color = np.random.randn(500), #set color equal to a variable
        color = feature_dataframe['Random Forest feature importances'].values,
        colorscale='Portland',
        showscale=True
    ),
    text = feature_dataframe['features'].values
)
data = [trace]

layout= go.Layout(
    autosize= True,
    title= 'Random Forest Feature Importance',
    hovermode= 'closest',
#     xaxis= dict(
#         title= 'Pop',
#         ticklen= 5,
#         zeroline= False,
#         gridwidth= 2,
#     ),
    yaxis=dict(
        title= 'Feature Importance',
        ticklen= 5,
        gridwidth= 2
    ),
    showlegend= False
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig,filename='scatter2010')

# Scatter plot 
trace = go.Scatter(
    y = feature_dataframe['Extra Trees  feature importances'].values,
    x = feature_dataframe['features'].values,
    mode='markers',
    marker=dict(
        sizemode = 'diameter',
        sizeref = 1,
        size = 25,
#       size= feature_dataframe['AdaBoost feature importances'].values,
        #color = np.random.randn(500), #set color equal to a variable
        color = feature_dataframe['Extra Trees  feature importances'].values,
        colorscale='Portland',
        showscale=True
    ),
    text = feature_dataframe['features'].values
)
data = [trace]

layout= go.Layout(
    autosize= True,
    title= 'Extra Trees Feature Importance',
    hovermode= 'closest',
#     xaxis= dict(
#         title= 'Pop',
#         ticklen= 5,
#         zeroline= False,
#         gridwidth= 2,
#     ),
    yaxis=dict(
        title= 'Feature Importance',
        ticklen= 5,
        gridwidth= 2
    ),
    showlegend= False
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig,filename='scatter2010')

# Scatter plot 
trace = go.Scatter(
    y = feature_dataframe['AdaBoost feature importances'].values,
    x = feature_dataframe['features'].values,
    mode='markers',
    marker=dict(
        sizemode = 'diameter',
        sizeref = 1,
        size = 25,
#       size= feature_dataframe['AdaBoost feature importances'].values,
        #color = np.random.randn(500), #set color equal to a variable
        color = feature_dataframe['AdaBoost feature importances'].values,
        colorscale='Portland',
        showscale=True
    ),
    text = feature_dataframe['features'].values
)
data = [trace]

layout= go.Layout(
    autosize= True,
    title= 'AdaBoost Feature Importance',
    hovermode= 'closest',
#     xaxis= dict(
#         title= 'Pop',
#         ticklen= 5,
#         zeroline= False,
#         gridwidth= 2,
#     ),
    yaxis=dict(
        title= 'Feature Importance',
        ticklen= 5,
        gridwidth= 2
    ),
    showlegend= False
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig,filename='scatter2010')

# Scatter plot 
trace = go.Scatter(
    y = feature_dataframe['Gradient Boost feature importances'].values,
    x = feature_dataframe['features'].values,
    mode='markers',
    marker=dict(
        sizemode = 'diameter',
        sizeref = 1,
        size = 25,
#       size= feature_dataframe['AdaBoost feature importances'].values,
        #color = np.random.randn(500), #set color equal to a variable
        color = feature_dataframe['Gradient Boost feature importances'].values,
        colorscale='Portland',
        showscale=True
    ),
    text = feature_dataframe['features'].values
)
data = [trace]

layout= go.Layout(
    autosize= True,
    title= 'Gradient Boosting Feature Importance',
    hovermode= 'closest',
#     xaxis= dict(
#         title= 'Pop',
#         ticklen= 5,
#         zeroline= False,
#         gridwidth= 2,
#     ),
    yaxis=dict(
        title= 'Feature Importance',
        ticklen= 5,
        gridwidth= 2
    ),
    showlegend= False
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig,filename='scatter2010')


이제 모든 featuer importance의 평균을 계산하고 데이터 프레임의 새로운 열로 그 값을 저장할것 입니다.

In [ ]:
# 평균 값을 포함하고 있는 새로운 column 생성

feature_dataframe['mean'] = feature_dataframe.mean(axis= 1) # axis = 1 computes the mean row-wise
feature_dataframe.head(3)

**Feature Importances 평균의 Plotly Barplot**

우리의 모든 classifier에 대한 feature importance 평균 값을 얻었으므로 그 값들을 이용해 Plotly bar plot을 다음과 같이 그릴 수 있다.

In [ ]:
y = feature_dataframe['mean'].values
x = feature_dataframe['features'].values
data = [go.Bar(
            x= x,
             y= y,
            width = 0.5,
            marker=dict(
               color = feature_dataframe['mean'].values,
            colorscale='Portland',
            showscale=True,
            reversescale = False
            ),
            opacity=0.6
        )]

layout= go.Layout(
    autosize= True,
    title= 'Barplots of Mean Feature Importance',
    hovermode= 'closest',
#     xaxis= dict(
#         title= 'Pop',
#         ticklen= 5,
#         zeroline= False,
#         gridwidth= 2,
#     ),
    yaxis=dict(
        title= 'Feature Importance',
        ticklen= 5,
        gridwidth= 2
    ),
    showlegend= False
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='bar-direct-labels')

# Second-Level Predictions from the First-level Output

**First-level output as new features**

첫번째 수준의 예측값을 얻은 후에 다음 classifier를 위한 training data로 사용될 feature의 새로운 set을 만드는것을 생각 할 수 있다. 아래 코드에 따라 이전 classifier로부터 얻은 첫 번째 수준의 예측값을 새로운 열로 가져오고 예측값를 이용해 다음의 classifier를 학습시킵니다.

In [ ]:
base_predictions_train = pd.DataFrame( {'RandomForest': rf_oof_train.ravel(),
     'ExtraTrees': et_oof_train.ravel(),
     'AdaBoost': ada_oof_train.ravel(),
      'GradientBoost': gb_oof_train.ravel()
    })
base_predictions_train.head()

**Correlation Heatmap of the Second Level Training set**

In [ ]:
data = [
    go.Heatmap(
        z= base_predictions_train.astype(float).corr().values ,
        x=base_predictions_train.columns.values,
        y= base_predictions_train.columns.values,
          colorscale='Viridis',
            showscale=True,
            reversescale = True
    )
]
py.iplot(data, filename='labelled-heatmap')

서로 상관성이 낮은 학습된 모델을 가지는것이 더 좋은 score를 생산한다는 장점이 있다는 기사와 Kaggle 경쟁 우승자 이야기가 꽤 많이 있습니다.

In [ ]:
x_train = np.concatenate(( et_oof_train, rf_oof_train, ada_oof_train, gb_oof_train, svc_oof_train), axis=1)
x_test = np.concatenate(( et_oof_test, rf_oof_test, ada_oof_test, gb_oof_test, svc_oof_test), axis=1)

이제 첫번째 수준의 train과 test 예측을 x_train 과 x_test로 연결시키고 결합 했으므로 두 번째 수준의 학습 모델을 적용 할 수 있습니다.

### XGBoost를 통한 두번째 수준의 학습 모델

여기서는 boosted tree learning model으로 매우 유명한 XGBoost를 선택했습니다. 이건 큰 scale의 boosted tree algorithms를 최적화 하도록 설계되어 있습니다. 알고리즘에 대한 정보를 얻고 싶다면 여기를 확인하세요.
[official documentation][1].

  [1]: https://xgboost.readthedocs.io/en/latest/

어쨋든 우리는 XGBclassifier를 호출하여 이것을 첫번째 수준의 train과 target에 fit 하고 학습된 모델을 test data를 예측하기 위해 다음과 같이 사용할 것입니다.:

In [ ]:
gbm = xgb.XGBClassifier(
    #learning_rate = 0.02,
 n_estimators= 2000,
 max_depth= 4,
 min_child_weight= 2,
 #gamma=1,
 gamma=0.9,                        
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread= -1,
 scale_pos_weight=1).fit(x_train, y_train)
predictions = gbm.predict(x_test)

model에서 사용되는 XGBoost parameters를 빠르게 훑어보자.:

**max_depth** :  당신의 tree가 어느정도의 깊이로 자라길 원하는가. 만약 너무 큰 값을 가진다면 overfitting의 가능성을 수반한다.

**gamma** : 트리의 리프 노드에서 추가 파티션을 작성하는 데 필요한 최소 손실 감소.크면 클수록 보수적인 알고리즘이 될 것입니다

**eta** : 오버 핏팅을 방지하기 위해 각 부스팅 단계에서 사용되는 축소되는 크기

**Producing the Submission file**

마침내 우리의 모든 첫번째 수준과 두번째 모델을 학습시키고 fit 했다. 이제 우리는 Titanic competition에 submission을 위해 적절한 형식으로 예측값을 다음과 같이 출력할 수 있다

In [ ]:
# Generate Submission File 
StackingSubmission = pd.DataFrame({ 'PassengerId': PassengerId,
                            'Survived': predictions })
StackingSubmission.to_csv("StackingSubmission.csv", index=False)

**개선을 위한 단계**

마지막으로, 위의 단계는 앙상블 스태커(ensemble stacker)를 제작하는 아주 간단한 방법을 보여줍니다. 당신은 아마도 가장 높은 수준의 Kaggle 대회에서의 앙상블은 괴상한 stacked classifiers의 조합은 물론이거니와 2번째 수준 이상의 stacking 수준도 보여준다는걸 확인 할 수 있을것 입니다.

점수를 높이기 위해 취할 수있는 추가 단계는:

 1. 최적의 parameter 값을 찾기 위해 모델을 학습 할 때 좋은 cross-validation strate 전략을 구현합니다.
 2. 학습을위한 다양한 base 모델을 사용하십시오. 결과가 더 관련성이 없으면 좋을수록 최종 점수가 높아집니다.

### Conclusion

이 notebook이 학습 모델을 스태킹하기위한 working script로 시작하는데 조금이라도 도움이 되도록 작성했습니다. 다시 한번 Faron과 Sina에게 감사의 마음을 전합니다.

스태킹이나 앙상블에 관한 다른 훌륭한 자료는 MLWave 웹 사이트에서 기사를 읽어볼 수 있습니다.: [Kaggle Ensembling Guide][1]. 

그럼 다음에 봐요, 안녕
  [1]: http://mlwave.com/kaggle-ensembling-guide/